In [155]:
import argparse
import base64
import json
import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
#import utils
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO
from keras.optimizers import Adam
import csv
import json
import cv2
import numpy as np
import matplotlib.image as mpimg
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline


from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

import os
import argparse
#import utils
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam


In [156]:
# Fix error with Keras and TensorFlow
import tensorflow as tf
tf.python.control_flow_ops = tf

In [157]:
DRIVING_CSV_LOG_PATH = "data/driving_log.csv"
DRIVING_IMAGES_PATH = "data/data/"

In [158]:
def image_process(data):
    
    #read image and convert
    img = cv2.imread(data[0])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    row,col,channels=img.shape

    #trucate sky part and dashboard part
    top=int(row*0.35)
    bottom = int(row*0.85)
    img = img[top:bottom,:]
    
    #resize
    img=cv2.resize(img,(220,66),interpolation=cv2.INTER_AREA)
    
    # change brightness
    # Note:HSV stands for hue, saturation, and value, and is also often called HSB (B for brightness). 
    percent = np.random.uniform(0.5,1.4)
    hsv = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    hsv[:,:,2] = hsv[:,:,2]*percent
    img=cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
    
    #getting steering angle
    steering_angle=float(data[1])
    
    #randomly filp center images
    if ("center" in data[0]):
        if np.random.randint(2)==1:
            img = cv2.flip(img, 1)
            steering_angle= -steering_angle
        
    return img, steering_angle
    
    
    

In [159]:
def load_csv():
    with open (DRIVING_CSV_LOG_PATH,'r') as file:
        reader = csv.reader(file)
        file.readline() #ignore header from csv file
        data=np.array([line for line in reader])
    return data

In [160]:
def normalize(data):
    a = 0.01
    b = 0.99
    color_min = 0.0
    color_max = 255.0
    return a + ((data - color_min) * (b-a))/(color_max - color_min)

In [161]:
def scale_and_normalize(img):
	img = img[::4,::2].copy()
	return img/127.5 - 1.0

In [162]:
def split_input(data):
	new_data = np.zeros([0, 2]) # Will be of shape(3*len(data), 2) because 3 images for one steering angle

	for i in range(0, len(data)):
		path_center_images = data[:,0][i].strip()
		path_left_images = data[:,1][i].strip()
		path_right_images = data[:,2][i].strip()
		steering_angle = float(data[:,3][i])
		new_row_center = [DRIVING_IMAGES_PATH + path_center_images, steering_angle]
		new_row_left = [DRIVING_IMAGES_PATH + path_left_images, steering_angle+0.2]
		new_row_right = [DRIVING_IMAGES_PATH + path_right_images, steering_angle-0.2]
		new_data = np.vstack([new_data, new_row_center])
		new_data = np.vstack([new_data, new_row_left])
		new_data = np.vstack([new_data, new_row_right])

	np.savetxt("expanded_driving_log.csv", new_data, delimiter=", ", fmt="%s")
	return new_data

In [163]:
def save_model(model):
	with open('./model.json', 'w') as outfile:
		json.dump(model.to_json(), outfile)

In [164]:
data=load_csv() #load the csv file
data = split_input(data) #create a new csv with modified csv
#print (data) #[['data/data/IMG/center_2016_12_01_13_30_48_287.jpg' '0.0']]

In [165]:
np.random.shuffle(data)
y_data= np.array(data[:,1],dtype=float)
X_train, X_val, y_train, y_val = train_test_split(data, y_data, test_size=0.1)

In [166]:
#print(X_train.shape, X_val.shape, y_train.shape, y_val.shape) #(21697, 2) (2411, 2) (21697,) (2411,)

In [167]:
print (len(X_train)) #21697
# print (X_train[500])
# print (y_train[500])

21697


In [168]:
def next_batch(X_train,y_train,batch_size):
    X=np.zeros((batch_size,66,220,3))
    y=np.zeros(batch_size)
    
    while True:
        for i in range (0,batch_size):
            index=np.random.randint(len(X_train))
            X[i],y[i]=image_process(X_train[index])
        yield X,y
    

In [169]:
def train_model(model,X_train,y_train,X_val,y_val,batch_size,nb_epoch):
    early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=2,verbose=0,mode='auto')
    checkpoint=ModelCheckpoint("model1.h5",monitor='val_loss',verbose=0,save_best_only=False,save_weights_only=False,mode='auto')
    
    model.fit_generator( generator=next_batch(X_train,y_train,batch_size),
    samples_per_epoch=20224,
    nb_epoch=nb_epoch,
    validation_data=next_batch(X_val,y_val,batch_size),
    nb_val_samples=len(X_val),
    callbacks=[early_stopping,checkpoint]
    )
    


In [170]:
# Use the model defined in Commai following repo:
# https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
def get_model():

	model = Sequential()
	adam = Adam(lr=0.001)

	# Nvidia
	model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(66, 220, 3), output_shape=(66, 220, 3)))
	model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal'))
	model.add(ELU())
	model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal'))
	model.add(ELU())
	model.add(Dropout(0.5))
	model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal'))
	model.add(ELU())
	model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init='he_normal'))
	model.add(Dropout(0.2))
	model.add(ELU())
	model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init='he_normal'))
	model.add(Flatten())
	model.add(ELU())
	model.add(Dense(1164, init='he_normal'))
	model.add(ELU())
	model.add(Dense(100, init='he_normal'))
	model.add(ELU())
	model.add(Dense(50, init='he_normal'))
	model.add(ELU())
	model.add(Dense(10, init='he_normal'))
	model.add(ELU())
	model.add(Dense(1, init='he_normal'))
	
	# Use the Adam optimizer to optimize the mean squared error
	model.compile(optimizer=adam, loss="mse")	

	return model


In [171]:
model = get_model()

In [173]:
train_model=train_model(model,X_train,y_train,X_val,y_val,256,5)

Epoch 1/5
20224/20224 [==============================] - 169s - loss: 0.1262 - val_loss: 0.0331
Epoch 2/5
20224/20224 [==============================] - 174s - loss: 0.0295 - val_loss: 0.0340
Epoch 3/5
20224/20224 [==============================] - 157s - loss: 0.0257 - val_loss: 0.0243
Epoch 4/5
20224/20224 [==============================] - 157s - loss: 0.0239 - val_loss: 0.0219
Epoch 5/5
20224/20224 [==============================] - 162s - loss: 0.0225 - val_loss: 0.0244


In [174]:
# # serialize model to JSON
# model_json = model.to_json()
# with open("model7.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# # model.save_weights("model7.h5")
# # print("Saved model to disk")

Saved model to disk


In [ ]:
#####

In [177]:
def save_model(model):
	with open('./model9.json', 'w') as outfile:
		json.dump(model.to_json(), outfile)

In [178]:
save_model(model)

In [150]:
#save_model(train_model)
from keras.models import load_model
json_string = model.to_json()
# with open('./model3.json', 'w') as outfile:
# 		json.dump(train_model.to_json(), outfile)

In [151]:
with open('./model4.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)

In [ ]:
def save_model2(train_model):
    

In [147]:
#python drive.py model.json
def save_model1(model, options):
    model.compile(optimizer=adam, loss="mse")

    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

def load_model(options):
    model = model_from_json(open(options['file_arch']).read())
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    model.load_weights(options['file_weight'])
    return model

In [148]:
save_model1(train_model,'a')

AttributeError: 'NoneType' object has no attribute 'compile'

In [ ]:
###############################33

In [4]:
def load_csv():
    with open (DRIVING_CSV_LOG_PATH,'r') as file:
        reader = csv.reader(file)
        file.readline() #ignore header from csv file
        data=np.array([line for line in reader])
    return data

In [5]:
loaded_csv=load_csv()
print(loaded_csv)

[['IMG/center_2016_12_01_13_30_48_287.jpg'
  ' IMG/left_2016_12_01_13_30_48_287.jpg'
  ' IMG/right_2016_12_01_13_30_48_287.jpg' ..., ' 0' ' 0' ' 22.14829']
 ['IMG/center_2016_12_01_13_30_48_404.jpg'
  ' IMG/left_2016_12_01_13_30_48_404.jpg'
  ' IMG/right_2016_12_01_13_30_48_404.jpg' ..., ' 0' ' 0' ' 21.87963']
 ['IMG/center_2016_12_01_13_31_12_937.jpg'
  ' IMG/left_2016_12_01_13_31_12_937.jpg'
  ' IMG/right_2016_12_01_13_31_12_937.jpg' ..., ' 0' ' 0' ' 1.453011']
 ..., 
 ['IMG/center_2016_12_01_13_46_38_846.jpg'
  ' IMG/left_2016_12_01_13_46_38_846.jpg'
  ' IMG/right_2016_12_01_13_46_38_846.jpg' ..., ' 0' ' 0' ' 1.388364']
 ['IMG/center_2016_12_01_13_46_38_922.jpg'
  ' IMG/left_2016_12_01_13_46_38_922.jpg'
  ' IMG/right_2016_12_01_13_46_38_922.jpg' ..., ' 0' ' 0' ' 1.377208']
 ['IMG/center_2016_12_01_13_46_38_947.jpg'
  ' IMG/left_2016_12_01_13_46_38_947.jpg'
  ' IMG/right_2016_12_01_13_46_38_947.jpg' ..., ' 0' ' 0' ' 1.374433']]


In [6]:
# columns = ['center','left', 'right', 'angle', 'throttle', 'break', 'speed']
# df_main_track = pd.read_csv(DRIVING_CSV_LOG_PATH, names=columns)
# df_main_edges = pd.read_csv(DRIVING_CSV_LOG_PATH", names=columns)
# df_main_udacity = pd.read_csv("main-track-udacity/driving_log.csv")
# df_main_good_quality = pd.read_csv("main-track-good-quality/driving_log.csv", names=columns)
# df_first = pd.read_csv("driving_log.csv", names=columns)

In [7]:
def split_input(data):
	new_data = np.zeros([0, 2]) # Will be of shape(3*len(data), 2) because 3 images for one steering angle

	for i in range(0, len(data)):
		path_center_images = data[:,0][i].strip()
		path_left_images = data[:,1][i].strip()
		path_right_images = data[:,2][i].strip()
		steering_angle = float(data[:,3][i])
		new_row_center = [DRIVING_IMAGES_PATH + path_center_images, steering_angle]
		new_row_left = [DRIVING_IMAGES_PATH + path_left_images, steering_angle+0.2]
		new_row_right = [DRIVING_IMAGES_PATH + path_right_images, steering_angle-0.2]
		new_data = np.vstack([new_data, new_row_center])
		new_data = np.vstack([new_data, new_row_left])
		new_data = np.vstack([new_data, new_row_right])

	np.savetxt("expanded_driving_log.csv", new_data, delimiter=", ", fmt="%s")
	return new_data
# df_main_udacity['angle'] = df_main_udacity['steering']
# path = '/Users/lacho/sdcnd/Self-Driving-Car-Simulator/main-track-udacity/'
# df_main_udacity['center'] = path + df_main_udacity['center'].str.strip()
# df_main_udacity['left'] = path + df_main_udacity['left'].str.strip()
# df_main_udacity['right'] = path + df_main_udacity['right'].str.strip()

In [8]:

# print(df_main_track.shape)
# print(df_main_edges.shape)
# print(df_main_udacity.shape)
# print(df_main_good_quality.shape)
# print(df_first.shape)

In [10]:
data=split_input(loaded_csv)

In [11]:
def scale(img):
    row,col,channel=img.shape
    top=int(row*0.3)
    bottom=int(row*0.8)
    img = img[top:bottom,:]
    img=cv2.resize(img,(220,66),interpolation=cv2.INTER_AREA)
    return img

In [ ]:
# def load_img(path):
#     try:
#         img=cv2.imread(path)
#         img = cv2.cvtColor(img,cv2.COLOR_BAYER_BGR2RGB)
#         img = scale(img)
#     except:
#         print ("Error loading file")
#         print (path)
#     return img

    

In [ ]:
# def data_generator(x,batch_size,iterations=1000):
#     for i in range(iterations):
#         imgs = np.array([load_img(x) for x in batch_size])
#         angles = np.array
    

In [13]:
y_data = np.array(data[:,1], dtype=float)
x_data = np.array(data[:,0])

print (x_data)


['data/data/IMG/center_2016_12_01_13_30_48_287.jpg'
 'data/data/IMG/left_2016_12_01_13_30_48_287.jpg'
 'data/data/IMG/right_2016_12_01_13_30_48_287.jpg' ...,
 'data/data/IMG/center_2016_12_01_13_46_38_947.jpg'
 'data/data/IMG/left_2016_12_01_13_46_38_947.jpg'
 'data/data/IMG/right_2016_12_01_13_46_38_947.jpg']


In [ ]:
X_train=[]
result_img= []
for i in range(len(x_data)):
    result_img = cv2.imread(x_data[i])
    X_train=np.append(X_train,result_img)

#plt.imshow(result_img[100])

In [ ]:
print (result_img)

In [ ]:

for i in range(len(x_data)):
    new = scale(result_img[i])
    X_train.append((X_train,new))

In [47]:
print (y_data)

[ 0.   0.2 -0.2 ...,  0.   0.2 -0.2]


In [50]:
print (x_data)

['data/data/IMG/center_2016_12_01_13_30_48_287.jpg'
 'data/data/ IMG/left_2016_12_01_13_30_48_287.jpg'
 'data/data/ IMG/right_2016_12_01_13_30_48_287.jpg' ...,
 'data/data/IMG/center_2016_12_01_13_46_38_947.jpg'
 'data/data/ IMG/left_2016_12_01_13_46_38_947.jpg'
 'data/data/ IMG/right_2016_12_01_13_46_38_947.jpg']
